In [10]:
!pip install optuna


In [24]:
!pip install pygam


   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/43.7 MB ? eta -:--:--
   -----------------

  You can safely remove it manually.


In [19]:
!pip install xgboost


   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/124.9 MB 840.2 kB/s eta 0:02:29
   ---------------------------------------- 0.8/124.9 MB 958.5 kB/s eta 0:02:10
   ---------------------------------------- 1.0/124.9 MB 898.8 kB/s eta 0:02:18
   ---------------------------------------- 1.0/124.9 MB 898.8 kB/s eta 0:02:18
   ---------------------------------------- 1.3/124.9 MB 849.7 kB/s eta 0:02:26
    --------------------------------------- 1.6/124.9 MB 830.6 kB/s eta 0:02:29
    --------------------------------------- 1.6/124.9 MB 830.6 kB/s eta 0:02:29
    --------------------------------------- 1.6/124.9 MB 830.6 kB/s eta 0:02:29
    --------------------------------------- 1.8/124.9 MB 694.4 kB/s eta 0:02:58
    --------------------------------------- 1.8/124.9 MB 694.4 kB/s

In [14]:
import pandas as pd

# Corrected file path
file_path = r"C:\Users\user\Downloads\archive (4)\DataCoSupplyChainDataset.csv"

# Load the CSV file
try:
    data = pd.read_csv(file_path, encoding='cp1252')
    print("File loaded successfully!")
except FileNotFoundError:
    print("Error: File not found. Please check the file path.")


File loaded successfully!


In [21]:
# Check the shape and first few rows of the dataset
print("Initial dataset shape:", data.shape)
print(data.head())

# Preprocessing
# Drop rows with missing values
data.dropna(inplace=True)
print("Shape after dropping missing values:", data.shape)  # Check shape after dropping


Initial dataset shape: (0, 53)
Empty DataFrame
Columns: [Type, Days for shipping (real), Days for shipment (scheduled), Benefit per order, Sales per customer, Delivery Status, Late_delivery_risk, Category Id, Category Name, Customer City, Customer Country, Customer Email, Customer Fname, Customer Id, Customer Lname, Customer Password, Customer Segment, Customer State, Customer Street, Customer Zipcode, Department Id, Department Name, Latitude, Longitude, Market, Order City, Order Country, Order Customer Id, order date (DateOrders), Order Id, Order Item Cardprod Id, Order Item Discount, Order Item Discount Rate, Order Item Id, Order Item Product Price, Order Item Profit Ratio, Order Item Quantity, Sales, Order Item Total, Order Profit Per Order, Order Region, Order State, Order Status, Order Zipcode, Product Card Id, Product Category Id, Product Description, Product Image, Product Name, Product Price, Product Status, shipping date (DateOrders), Shipping Mode]
Index: []

[0 rows x 53 col

In [17]:
import pandas as pd
import optuna
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  # Assuming RandomForest for this example
from sklearn.metrics import mean_squared_error



# Preprocess the dataset (simplified)
X = data[["Days for shipping (real)", "Days for shipment (scheduled)", "Benefit per order", "Sales per customer", "Late_delivery_risk"]]
y = data["Order Profit Per Order"]

# Define objective function
def objective(trial):
    # Suggest hyperparameters for tuning
    n_estimators = trial.suggest_int("n_estimators", 10, 100)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    
    # Split data
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Model training
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    # Predictions and evaluation
    y_pred = model.predict(X_valid)
    error = mean_squared_error(y_valid, y_pred)
    
    return error

# Run Optuna optimization with multiple seeds
results = []
for seed in range(5):
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=seed))
    study.optimize(objective, 10)  # Customize number of trials as needed
    results.append(study.best_value)

# Calculate median and variance
median_result = np.median(results)
variance_result = np.var(results)

print("Median of best scores:", median_result)
print("Variance of best scores:", variance_result)


[I 2025-01-24 21:52:01,206] A new study created in memory with name: no-name-6c350e79-51fd-42f5-ab3d-cad1297c2a76
[I 2025-01-24 21:52:32,266] Trial 0 finished with value: 0.3014098219020876 and parameters: {'n_estimators': 59, 'max_depth': 8, 'min_samples_split': 7}. Best is trial 0 with value: 0.3014098219020876.
[I 2025-01-24 21:52:49,931] Trial 1 finished with value: 9.518289224465947 and parameters: {'n_estimators': 59, 'max_depth': 6, 'min_samples_split': 7}. Best is trial 0 with value: 0.3014098219020876.
[I 2025-01-24 21:53:10,836] Trial 2 finished with value: 0.16894949086489097 and parameters: {'n_estimators': 49, 'max_depth': 10, 'min_samples_split': 10}. Best is trial 2 with value: 0.16894949086489097.
[I 2025-01-24 21:53:27,270] Trial 3 finished with value: 0.07203018359191654 and parameters: {'n_estimators': 44, 'max_depth': 9, 'min_samples_split': 6}. Best is trial 3 with value: 0.07203018359191654.
[I 2025-01-24 21:53:51,609] Trial 4 finished with value: 0.03048414779923

Median of best scores: 0.04367127461668552
Variance of best scores: 0.00039116597644002737


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Preprocessing
data.dropna(inplace=True)  # Example: drop rows with missing values

# Define features and target
columns_to_drop = ['Delivery Status', 'Customer Email', 'Customer Fname', 'Customer Id', 'Customer Lname', 'Order Status']
X = data.drop(columns=[col for col in columns_to_drop if col in data.columns], errors='ignore')  # Drop columns safely
y = data['Days for shipping (real)']  # Ensure this column exists

# Confirm the presence of features
print("Features shape:", X.shape)
print("Target shape:", y.shape)

# Splitting the data into train and test
if not X.empty and not y.empty:  # Ensure both are non-empty
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Function to optimize using Optuna
    def objective(trial):
        model_type = trial.suggest_categorical("model_type", ["XGBoost", "SVM", "RandomForest"])
        
        if model_type == "XGBoost":
            params = {
                'max_depth': trial.suggest_int('max_depth', 1, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
                'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                'objective': 'reg:squarederror'
            }
            model = xgb.XGBRegressor(**params)
            
        elif model_type == "SVM":
            params = {
                'C': trial.suggest_float('C', 0.1, 10),
                'epsilon': trial.suggest_float('epsilon', 0.01, 1),
                'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf'])
            }
            model = SVR(**params)
            
        elif model_type == "RandomForest":
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                'max_depth': trial.suggest_int('max_depth', 1, 10),
            }
            model = RandomForestRegressor(**params)
        
        # Fit the model
        model.fit(X_train, y_train)
        
        # Predictions
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        return rmse  # Optimize for RMSE

    # Run Optuna study
    study = optuna.create_study(direction='minimize')
    for seed in range(5):
        print(f"Running optimization for seed {seed}")
        np.random.seed(seed)
        study.optimize(objective, n_trials=100)

    # Report median and variance of results
    results = [study.best_value for _ in range(5)]  # Assuming each run gives one best value
    median = np.median(results)
    variance = np.var(results)

    print(f"Median RMSE: {median}, Variance: {variance}")
else:
    print("Features or target are empty. Check your data preprocessing.")


Features shape: (0, 47)
Target shape: (0,)
Features or target are empty. Check your data preprocessing.


In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import optuna
from pygam import LinearGAM



# Define features and target
columns_to_drop = ['Delivery Status', 'Customer Email', 'Customer Fname', 'Customer Id', 'Customer Lname', 'Order Status']
X = data.drop(columns=[col for col in columns_to_drop if col in data.columns], errors='ignore')
y = data['Days for shipping (real)']

# Check if we still have data for training
print("Features shape:", X.shape)
print("Target shape:", y.shape)

# Ensure we have enough samples to split
if X.shape[0] > 0 and y.shape[0] > 0:
    # Splitting the data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Function to optimize using Optuna
    def objective(trial):
        model_type = trial.suggest_categorical("model_type", ["XGBoost", "SVM", "RandomForest", "GAM"])
        
        if model_type == "XGBoost":
            params = {
                'max_depth': trial.suggest_int('max_depth', 1, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
                'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                'objective': 'reg:squarederror'
            }
            model = xgb.XGBRegressor(**params)
            
        elif model_type == "SVM":
            params = {
                'C': trial.suggest_float('C', 0.1, 10),
                'epsilon': trial.suggest_float('epsilon', 0.01, 1),
                'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf'])
            }
            model = SVR(**params)
            
        elif model_type == "RandomForest":
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 500),
                'max_depth': trial.suggest_int('max_depth', 1, 10),
            }
            model = RandomForestRegressor(**params)
            
        elif model_type == "GAM":
            model = LinearGAM()  # Adjust parameters as necessary
        
        # Fit the model
        model.fit(X_train, y_train)
        
        # Predictions
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        return rmse  # Optimize for RMSE

    # Run Optuna study for different seeds and collect results
    seed_results = {}

    for seed in range(5):
        print(f"Running optimization for seed {seed}")
        np.random.seed(seed)
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=100)
        
        # Store the best RMSE for this seed
        seed_results[seed] = study.best_value

    # Reporting median and variance
    median_rmse = np.median(list(seed_results.values()))
    variance_rmse = np.var(list(seed_results.values()))

    print(f"Results for 5 seeds: {seed_results}")
    print(f"Median RMSE: {median_rmse}, Variance: {variance_rmse}")
else:
    print("Not enough data to perform train-test split.")


Features shape: (0, 47)
Target shape: (0,)
Not enough data to perform train-test split.


In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

data = pd.read_csv(file_path, encoding='cp1252')
# Assuming the dataset has already been preprocessed:
# Select relevant features and the target variable
X = data.drop(columns=['Days for shipping (real)'])  # Drop the target variable and any non-feature columns
y = data['Days for shipping (real)']

# Handling categorical variables if any (example: using one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost testing function
def xgboost_testing(X_train, y_train, X_test, y_test, params=None):
    """
    Trains an XGBoost model and evaluates it on the test dataset.

    Parameters:
    X_train (DataFrame): Training features
    y_train (Series): Training target
    X_test (DataFrame): Testing features
    y_test (Series): Testing target
    params (dict): Hyperparameters for the XGBoost model

    Returns:
    dict: A dictionary containing RMSE and R² scores
    """
    
    # If no params are provided, use default values
    if params is None:
        params = {
            'max_depth': 3,
            'learning_rate': 0.1,
            'n_estimators': 100,
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse'
        }

    # Create an XGBoost Regressor
    model = xgb.XGBRegressor(**params)

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Calculate R² score
    r2 = r2_score(y_test, y_pred)

    # Report the results
    results = {
        'RMSE': rmse,
        'R²': r2
    }

    print(f"XGBoost Testing Results:\nRMSE: {rmse}\nR²: {r2}")

    return results

# Run the XGBoost testing
results = xgboost_testing(X_train, y_train, X_test, y_test)

# Print the results
print(results)


MemoryError: Unable to allocate 11.1 GiB for an array with shape (180519, 65752) and data type bool

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load your data

# Identify numerical and categorical columns
num_cols = data.select_dtypes(include=['float64', 'int64']).columns
cat_cols = data.select_dtypes(include=['object']).columns

# Print identified columns for verification
print("Numerical Columns:", num_cols.tolist())
print("Categorical Columns:", cat_cols.tolist())

# Check for non-numeric values in numerical columns
for col in num_cols:
    print(f"Unique values in {col}:", data[col].unique())

# Convert non-numeric data to NaN in numerical columns, if necessary
data[num_cols] = data[num_cols].apply(pd.to_numeric, errors='coerce')

# Impute missing values for numerical columns
num_imputer = SimpleImputer(strategy='mean')
data[num_cols] = num_imputer.fit_transform(data[num_cols])

# Impute missing values for categorical columns
cat_imputer = SimpleImputer(strategy='most_frequent')
data[cat_cols] = cat_imputer.fit_transform(data[cat_cols])

# Check the DataFrame shape after imputation
print("Data shape after imputation:", data.shape)

# Select relevant features and the target variable
if 'Days for shipping (real)' in data.columns:
    X = data.drop(columns=['Days for shipping (real)'])  # Drop the target variable
    y = data['Days for shipping (real)']
else:
    raise ValueError("Target variable 'Days for shipping (real)' not found in the DataFrame.")

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the SVM model
model = SVR()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


In [ ]:

from sklearn.model_selection import train_test_split

# First split: Initial train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Second split: Splitting train into train and validation for hyperparameter tuning
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape}, Validation set size: {X_valid.shape}, Test set size: {X_test.shape}")
